<div style="padding:20px;color:white;margin:0;font-size:35px;font-family:arial;text-align:center;display:fill;border-radius:5px; letter-spacing: 2px; background-color:#257CCB;overflow:hidden"><b>INTEGRATED CONTINUOUS ASSESSMENT</b></div>

<a id="1.1"></a>
<h2 style="font-family:arial;text-align:center; font-size: 32px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: centre; letter-spacing: 5px; color:#257CCB; background-color: #ffffff;"> <b>DATA VISUALISATION + MACHINE LEARNING</b></h2>

**_Student:_** Bárbara Abreu Costa 2023099

**_Dashboard SuperStore:_** http://127.0.0.1:8050/

In [1]:
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import json

# Load data
file_path = 'Superstore.xlsx'
df = pd.read_excel(file_path)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Calculate required metrics
total_sales = df['Sales'].sum()
total_profit = df['Profit'].sum()
total_order = df['Order ID'].nunique()
total_item = df['Quantity'].sum()

# Format large numbers for display
def format_number(number):
    if number >= 10**6:
        return f'{number / 10**6:.1f}M'
    elif number >= 10**3:
        return f'{number / 10**3:.1f}K'
    else:
        return f'{number:.0f}'

total_sales_formatted = format_number(total_sales)
total_profit_formatted = format_number(total_profit)
total_order_formatted = format_number(total_order)
total_item_formatted = format_number(total_item)

styles = {
    'main_div': {'font-family': 'Arial, sans-serif', 'padding': '20px', 'background-color': 'white'},
    'title': {'font-size': '40px', 'margin-bottom': '20px', 'margin-top': '20px','text-align': 'center', 'color': '#257CCB'},
    'metrics_box': {'display':'inline-block', 'justify-content': 'center', 'width': '160px', 'height': '100px',
                    'margin-right': '100px','margin-left': '100px', 'padding': '20px',
                    'background-color': '#257CCB', 'color': 'white', 'text-align': 'center'},
    'metric_value': {'font-size': '24px', 'font-weight': 'bold', 'margin-bottom': '20px', 'background-color': '#257CCB'},
    'metric_label': {'font-size': '16px', 'margin-top': 'px', 'background-color': '#257CCB'}
}

df['Order Date'] = pd.to_datetime(df['Order Date'],format="%d-%m-%Y")
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format="%d-%m-%Y")
df['Order Month'] = df['Order Date'].dt.month
df['Order Year'] = df['Order Date'].dt.year
df['Order Year'].unique()
df['Month_Year'] = df['Order Date'].dt.to_period('M')

# Sales over the years
grouped_data = df.groupby(['Order Year'])['Sales'].sum()

# Sesanal sales over the years
agg_df = df.groupby(['Order Year', 'Order Month'])['Sales'].sum().reset_index()

# Top 10 Selling States
states_info = df.groupby("State", as_index=False).agg(
    Sales = ("Sales", "sum"),
    Profit = ("Profit", "sum")
)
states_info = states_info.sort_values(by = ['Sales'], ascending = False).head(10)

# Top 10 Selling Cities
cities_info = df.groupby("City", as_index=False).agg(
    sales = ("Sales", "sum"),
    profit = ("Profit", "sum")
)
top_Sales_city = cities_info[['City' ,'sales']].sort_values(by = ["sales"] ,ascending= False).head(10)

# Top Selling Products
Products_info = cities_info = df.groupby("Product Name", as_index=False).agg(
    Sales = ("Sales", "sum"),
    Profit = ("Profit", "sum")
)
Top_Salling_products = Products_info[["Product Name", "Sales"]].sort_values(by=['Sales'], ascending = False).head(10)

# Reading dataset GeoJSON
with open('us-states.json') as f:
    us_states_geojson = json.load(f)

SalesUSA = df.groupby('State')['Sales'].sum().sort_values(ascending=False).reset_index()

# Creating the graphics
fig1 = px.bar(df, x="Region", y="Sales", color="Category", barmode="group")
fig1.update_layout(hoverlabel_font_size=20)
fig1.update_layout(
    title={
        'text': "SALES BY REGION AND CATEGORY",
        'x': 0.5,  # Posição centralizada horizontalmente
        'y': 0.95,  # Posição ligeiramente abaixo do topo
        'xanchor': 'center',  # Alinhamento horizontal centralizado
        'yanchor': 'top'  # Alinhamento vertical no topo
    }
)

options = list(df['Category'].unique())
options.append('All Categories')

fig2 = px.line(grouped_data, x=grouped_data.index.astype(str), y=grouped_data.values, markers=True, line_shape='linear', title='SALES OVER THE YEARS')
fig2.update_xaxes(title='Year')
fig2.update_yaxes(title_text='Sale Amount')
fig2.update_layout(hoverlabel_font_size=20)

fig3 = px.line(agg_df, x='Order Month', y='Sales', color='Order Year', title='SEASONAL SALES OVER THE YEARS',
              labels={'Order Year': 'Year', 'Sales': 'Sum of Sales', 'Order Month': 'Month'},markers=True)

fig3.update_traces(line=dict(width=1.5))
fig3.update_layout(hoverlabel_font_size=20)

fig4 = px.bar(top_Sales_city,x='City', y='sales', title='🔝TOP 10 SALLING CITIES')
fig4.update_layout(hoverlabel_font_size=20)

fig5 = px.treemap(states_info,
                 title = '🔝 TOP 10 SELLING STATES',
                 path=['State','Sales'], 
                 values = 'Sales',
                 width=1200, height=400,
                 hover_data = ['State'], 
)

fig5.update_layout(width=1200, height=600,
                  paper_bgcolor='white',
                  plot_bgcolor='white'
                  )
fig5.update_traces(root_color='white')
fig5.update_traces(textfont=dict(size=16))
fig5.update_traces(hoverlabel_font_size=20)

# Top 10 Selling_products
fig6 = px.bar(Top_Salling_products, x='Sales', 
             y='Product Name', 
             title='🔝 TOP 10 SELLING PRODUCT')
fig6.update_layout(yaxis = {"categoryorder":"total ascending"})
fig6.update_traces(hoverlabel_font_size=20)

fig7 = fig = px.sunburst(
    df,
    path=["Category", "Sub-Category"],
    values="Sales",
    color="Sales",
    color_continuous_scale="RdBu",
    color_continuous_midpoint=2.1,
    width=700,
    height=700
)

fig7.update_traces(hovertemplate="<b>Sales: $%{value:.2f}")
 
fig7.update_layout(
    title="SALES OVERVIEW",
    title_font={"size": 16},  # Ajuste o tamanho do título conforme desejado
    legend=dict(font=dict(size=20)),  # Ajuste o tamanho da fonte da legenda
    font=dict(size=16)  # Ajuste o tamanho da fonte do texto dentro do gráfico

)

fig8 = px.choropleth(SalesUSA, 
                    geojson=us_states_geojson,
                    locations='State',   # Nombre de la columna en el DataFrame
                    featureidkey="properties.name",
                    color= 'Sales',   # Nombre de la columna en el DataFrame
                    color_continuous_scale='Viridis',  # Elige una paleta de colores
                   
                    )

fig7.update_layout(
    hoverlabel=dict(font=dict(size=25))

)

# Atualização do layout com o subtítulo
fig8.update_layout(
    title={
        'text': "SALES VIEW IN THE MAP<br><span style='font-size: 12px;'>🔍 HOVER OVER EACH STATE TO SEE THE SALES AMOUNT</span><br>",
        'y':0.95,  # Define a posição vertical do título
        'x':0.5,  # Define a posição horizontal do título
        'xanchor': 'center',  # Define o ponto de referência horizontal do título
        'yanchor': 'top',  # Define o ponto de referência vertical do título
        'font_size': 16  # Define o tamanho do texto do título
    },
    showlegend=False,
    geo_scope='usa'
)

fig8.update_traces(hovertemplate='%{location}<br>Sales: $%{z:,.2f}<extra></extra>')
fig8.update_layout(hoverlabel_font_size=20)


# Layout com métricas e gráficos
app.layout = html.Div(style=styles['main_div'], children=[
    html.H1(style=styles['title'], children='SUPERSTORE ANALYSIS'),
   
    # Metrics boxes
    html.Div([
        html.Div([
            html.Div([
                html.Div('TOTAL SALES', style=styles['metric_label']),
                html.Div(total_sales_formatted, style=styles['metric_value']),
            ], style=styles['metrics_box']),
            
            html.Div([
                html.Div('TOTAL PROFIT', style=styles['metric_label']),
                html.Div(total_profit_formatted, style=styles['metric_value']),
            ], style=styles['metrics_box']),
            
            html.Div([
                html.Div('TOTAL ORDER', style=styles['metric_label']),
                html.Div(total_order_formatted, style=styles['metric_value']),
            ], style=styles['metrics_box']),
            
            html.Div([
                html.Div('TOTAL ITEM', style=styles['metric_label']),
                html.Div(total_item_formatted, style=styles['metric_value']),
            ], style=styles['metrics_box']),
        ], style={'margin-bottom': '20px'})  # Adiciona margem abaixo dos boxes de métricas
    ]),
    
    
    # Gráficos usando dbc.Row e dbc.Col
    dbc.Row([
        dbc.Col(
            dcc.Graph(id='example-graph2', figure=fig2),
            width={"size": 6, "offset": 0}  # Tamanho da coluna e offset
        ),
        dbc.Col(
            dcc.Graph(id='example-graph3', figure=fig3),
            width={"size": 6, "offset": 0}  # Tamanho da coluna e offset
        ),
    ]),
    
    html.Div(children='''
        Choose the category ⬇️
    ''', style={'color': 'black', 'text-transform': 'uppercase'}),
    html.Div(id="text"),
    
    # Aqui você pode adicionar os gráficos existentes do seu aplicativo Dash
    dcc.Dropdown(options, value='All Categories', id='All Categories'),
    
    dcc.Graph(
        id='example-graph',
        figure=fig1),
    
    dcc.Graph(
        id='example-graph4',
        figure=fig4),
    
    dcc.Graph(
        id='example-graph5',
        figure=fig5),
    
    dcc.Graph(
        id='example-graph6',
        figure=fig6),
    
    dbc.Row([
        dbc.Col(
            dcc.Graph(id='example-graph7', figure=fig7),
            width={"size": 6, "offset": 0}
        ),
        dbc.Col(
            dcc.Graph(id='example-graph8', figure=fig8),
            width={"size": 6, "offset": 0}            
            
        ),
    ]),
    
])

@app.callback(
    Output('example-graph', 'figure'),
    Input('All Categories', 'value')
)
def update_output(value):
    if value == "All Categories":
        fig1 = px.bar(df, x="Region", y="Sales", color="Category", barmode="group")
    else:
        filtered_table = df.loc[df['Category'] == value, :]
        fig1 = px.bar(filtered_table, x="Region", y="Sales", color="Category", barmode="group")

    return fig1

if __name__ == '__main__':
    app.run_server(debug=False)